<a href="https://colab.research.google.com/github/su-anshu/Amazon/blob/main/Amazon_Highlighter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
pip install flask pymupdf

In [8]:
!pip install flask pyngrok pymupdf

In [9]:
!ngrok authtoken 2FWfyS2dYI0iGe2Ac7DjveNwZID_3iok8SoASeqtABzrNe7Wu

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [10]:
import fitz  # PyMuPDF
from flask import Flask, request, send_file, render_template
from pyngrok import ngrok
import os
import threading

# Setup Flask
app = Flask(__name__, template_folder="/content/templates")
UPLOAD_FOLDER = "/content/uploads"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

# PDF Processing Function
def highlight_large_qty(pdf_path, output_path):
    doc = fitz.open(pdf_path)

    for page in doc:
        text_blocks = page.get_text("blocks")
        qty_found = False

        for block in text_blocks:
            x0, y0, x1, y1, text, *_ = block

            if "Qty" in text:
                qty_found = True
                continue

            if qty_found:
                qty_found = False
                values = text.split()

                for val in values:
                    if val.isdigit() and int(val) > 1:
                        highlight_box = fitz.Rect(x0, y0, x1, y1)
                        page.draw_rect(highlight_box, color=(1, 0, 0), fill_opacity=0.4)

    doc.save(output_path)

# Serve the HTML Page
@app.route("/", methods=["GET"])
def index():
    return render_template("index.html")  # Loads the frontend

# Handle File Upload
@app.route("/upload", methods=["POST"])
def upload_file():
    if "file" not in request.files:
        return "No file uploaded", 400

    file = request.files["file"]
    if file.filename == "":
        return "No selected file", 400

    input_pdf = os.path.join(UPLOAD_FOLDER, file.filename)
    output_pdf = os.path.join(UPLOAD_FOLDER, "highlighted_" + file.filename)

    file.save(input_pdf)
    highlight_large_qty(input_pdf, output_pdf)

    return send_file(output_pdf, as_attachment=True)

# Start Flask in a Background Thread
def run_flask():
    app.run(port=5000, debug=True, use_reloader=False)

threading.Thread(target=run_flask, daemon=True).start()

# Expose Flask Server via ngrok
public_url = ngrok.connect(5000).public_url
print(f"🚀 Open this link: {public_url}")


 * Serving Flask app '__main__'
 * Debug mode: on


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


🚀 Open this link: https://9737-35-230-93-165.ngrok-free.app


In [6]:
!pkill -9 ngrok